In [1]:
import numpy as np
import time


index = np.load("/home/jovyan/Sample_Based_Extension/WUSTL/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9998827239867143,
        0.9600649800293866, array([1, 1, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9996984331086939,
        0.9305545272093338, array([3, 3, 3, ..., 3, 3, 3])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.9984796002563319,
        0.7518383136072314, array([5, 5, 5, ..., 5, 5, 5])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9999916231419081,
        0.9951091814583067, array([2, 2, 2, ..., 2, 2, 2])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9998701586995765,
        0.939002641264322, array([1, 1, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9971518682487759,
        0.8894642081911899, array([3, 3, 3, ..., 3, 3, 3])],
       ['FGSM', 0.3, 'Def

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    # ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1 and attack_name!= "CaFA":
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

unique_labels, counts = np.unique(y_label_0_1, return_counts=True)
print(unique_labels, counts)
valid_labels = unique_labels[counts > 5]
mask = np.isin(y_label_0_1, valid_labels)
x_test_adv_0_1 = x_test_adv_0_1[mask]
y_label_0_1 = y_label_0_1[mask]
print(np.unique(y_label_0_1, return_counts = True))

[ 1  2  3  4  5  6  7  8  9 10 11] [955054     14 479520     19   4005 467864      3   1083     52   2408
      2]
(array([ 1,  2,  3,  4,  5,  6,  8,  9, 10]), array([955054,     14, 479520,     19,   4005, 467864,   1083,     52,
         2408]))


In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '8' encode 6
defense '9' encode 7
defense '10' encode 8


In [5]:
# Collect 10% of eps = 0.1 data
from sklearn.model_selection import train_test_split

_, X_10, _ , y_10 = train_test_split(x_test_adv_0_1, y_label_0_1, test_size=0.1, stratify=y_label_0_1, random_state=42)


In [6]:
X_10.shape

(191002, 42)

In [7]:
from xgboost import XGBClassifier

print('xgb')
xgb10 = XGBClassifier()
xgb10.fit(X_10, y_10)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf10 = RandomForestClassifier()
rf10.fit(X_10, y_10)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt10 = DecisionTreeClassifier()
# dt10.fit(X_10, y_10)

xgb
RF


RandomForestClassifier()

In [8]:
y_test = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [9]:
import time
import numpy as np




from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = xgb10.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/XGB_ExcludeCaFA/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[1 3 6] [     3      3 238747]
(3, 42) (3,)
Save 1 to baseline_Def1.npy
(3, 42) (3,)
Save 3 to baseline_Def3.npy
(238747, 42) (238747,)
Save 6 to baseline_Def6.npy
Execution Time: 0.497049 seconds
(716259,)
[ 1  3  4  5  6  9 10] [429567 269504   1181   3713   9719   1782    793]
(429567, 42) (429567,)
Save 1 to BIM_Def1.npy
(269504, 42) (269504,)
Save 3 to BIM_Def3.npy
(1181, 42) (1181,)
Save 4 to BIM_Def4.npy
(3713, 42) (3713,)
Save 5 to BIM_Def5.npy
(9719, 42) (9719,)
Save 6 to BIM_Def6.npy
(1782, 42) (1782,)
Save 9 to BIM_Def9.npy
(793, 42) (793,)
Save 10 to BIM_Def10.npy
Execution Time: 1.215763 seconds
(716259,)
[ 1  3  4  5  6  8  9 10] [445867 235624      6  14421  13573   5714    194    860]
(445867, 42) (445867,)
Save 1 to FGSM_Def1.npy
(235624, 42) (235624,)
Save 3 to FGSM_Def3.npy
(6, 42) (6,)
Save 4 to FGSM_Def4.npy
(14421, 42) (14421,)
Save 5 to FGSM_Def5.npy
(13573, 42) (13573,)
Save 6 to FGSM_Def6.npy
(5714, 42) (5714,)
Save 8 to FGSM_Def8.npy
(194, 42) (194,)

In [10]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

    y_pred = rf10.predict(X)
    print(y_pred.shape)
    y_pred = label_encoder.inverse_transform(y_pred)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/RF_ExcludeCaFA/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(238753,)
[3 6] [     2 238751]
(2, 42) (2,)
Save 3 to baseline_Def3.npy
(238751, 42) (238751,)
Save 6 to baseline_Def6.npy
Execution Time: 2.298150 seconds
(716259,)
[ 1  3  5  6 10] [225371 488053   1544   1284      7]
(225371, 42) (225371,)
Save 1 to BIM_Def1.npy
(488053, 42) (488053,)
Save 3 to BIM_Def3.npy
(1544, 42) (1544,)
Save 5 to BIM_Def5.npy
(1284, 42) (1284,)
Save 6 to BIM_Def6.npy
(7, 42) (7,)
Save 10 to BIM_Def10.npy
Execution Time: 4.953443 seconds
(716259,)
[ 1  3  5  6  9 10] [443591 243366  10814  15958     10   2520]
(443591, 42) (443591,)
Save 1 to FGSM_Def1.npy
(243366, 42) (243366,)
Save 3 to FGSM_Def3.npy
(10814, 42) (10814,)
Save 5 to FGSM_Def5.npy
(15958, 42) (15958,)
Save 6 to FGSM_Def6.npy
(10, 42) (10,)
Save 9 to FGSM_Def9.npy
(2520, 42) (2520,)
Save 10 to FGSM_Def10.npy
Execution Time: 3.813037 seconds
(716259,)
[ 1  3  5  6 10] [225371 488053   1544   1284      7]
(225371, 42) (225371,)
Save 1 to PGD_Def1.npy
(488053, 42) (488053,)
Save 3 to PGD_Def3.npy
(

In [11]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/WUSTL/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/WUSTL/y_test.npy')

#     y_pred = dt10.predict(X)
#     print(y_pred.shape)
#     y_pred = label_encoder.inverse_transform(y_pred)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ExcludeCaFA/WUSTL_Input10/x_test_adv_{attack_name}_Def{class_num}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/DT_ExcludeCaFA/WUSTL_Input10/y_test_adv_{attack_name}_Def{class_num}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

In [12]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}10")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/WUSTL/WUSTL_Dynamite_Selection_Data/{m_name}_ExcludeCaFA/WUSTL_Input10/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [13]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input10_ExcludeCaFA.csv")

In [14]:
all_attack_list

0       1   2       3      4      5       6   7     8     9      10  \
0   XGB10       3   0       3      0      0  238747   0     0     0      0   
1   XGB10  429567   0  269504   1181   3713    9719   0     0  1782    793   
2   XGB10  445867   0  235624      6  14421   13573   0  5714   194    860   
3   XGB10  429567   0  269504   1181   3713    9719   0     0  1782    793   
4   XGB10      79   0    6033      0  10332  687682   0  3514   123   8496   
5   XGB10  144477   0  469704  50038    980   47980   0   228  1839   1013   
6   XGB10       9   0      18      0      0  716232   0     0     0      0   
7   XGB10   55254   0  322175      0     48  327714   0     0     0  11068   
8   XGB10    1891   0  706503      4   1721    5787   0     0   353      0   
9   XGB10  398711   0  258057     36   1064   56166   0    12   404   1809   
10   RF10       0   0       2      0      0  238751   0     0     0      0   
11   RF10  225371   0  488053      0   1544    1284   0     0     0      7   
12   RF10  443591   0  243366      0  10814   15958   0     0    10   2520   
13   RF10  225371   0  488053      0   1544    1284   0     0     0      7   
14   RF10      65   0    6131     36   8767  687753   0  3740   165   9602   
15   RF10  159020   0  544889      0     23   12277   0     0     0     50   
16   RF10       0   0      17      0      0  716242   0     0     0      0   
17   RF10     161   0  522638      0      0  193460   0     0     0      0   
18   RF10    1674   0  709614      0      0    4971   0     0     0      0   
19   RF10  398465   0  256280      0    593   60894   0     0     0     27   
20   DT10       0   0       0      0      0       0   0     0     0      0   
21   DT10       0   0       0      0      0       0   0     0     0      0   
22   DT10       0   0       0      0      0       0   0     0     0      0   
23   DT10       0   0       0      0      0       0   0     0     0      0   
24   DT10       0   0       0      0      0       0   0     0     0      0   
25   DT10       0   0       0      0      0       0   0     0     0      0   
26   DT10       0   0       0      0      0       0   0     0     0      0   
27   DT10       0   0       0      0      0       0   0     0     0      0   
28   DT10       0   0       0      0      0       0   0     0     0      0   
29   DT10       0   0       0      0      0       0   0     0     0      0   

    11      12  
0    0  238753  
1    0  716259  
2    0  716259  
3    0  716259  
4    0  716259  
5    0  716259  
6    0  716259  
7    0  716259  
8    0  716259  
9    0  716259  
10   0  238753  
11   0  716259  
12   0  716259  
13   0  716259  
14   0  716259  
15   0  716259  
16   0  716259  
17   0  716259  
18   0  716259  
19   0  716259  
20   0       0  
21   0       0  
22   0       0  
23   0       0  
24   0       0  
25   0       0  
26   0       0  
27   0       0  
28   0       0  
29   0       0